<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/code_commune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!jupyter nbconvert --to pdf code commune.ipynb

[NbConvertApp] WARNING | pattern 'code' matched no files
[NbConvertApp] WARNING | pattern 'commune.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equiva

In [2]:
!pip install nbconvert

In [3]:
!pip install reportlab
!pip install pdfkit
!pip install tqdm
!pip install dateparser
!pip install unidecode

import pandas as pd
import numpy as np
from datetime import datetime
import locale
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table
import pdfkit
import tabulate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import html
import dateparser
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
from tqdm import tqdm
from unidecode import unidecode

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Commune depuis 1943

In [4]:
df = pd.read_csv('v_commune_depuis_1943.csv')
df.columns

Index(['COM', 'TNCC', 'NCC', 'NCCENR', 'LIBELLE', 'DATE_DEBUT', 'DATE_FIN'], dtype='object')

**Description des colonnes** : <br>
- La colonne 'COM' correspond au Code commune <br>
- La colonne 'TNCC' correspond au Type de nom en clair <br>
- La colonne 'NCC' correspond au Nom en clair (majuscules) <br>
- La colonne 'NCCENR' correspond au Nom en clair (typographie riche) <br>
- La colonne 'LIBELLE' correspond au Nom en clair (typographie riche) avec article <br>
- La colonne 'DATE_DEBUT' correspond au Date de début du couple code*libellé de commune (AAAA-MM-JJ) <br>
- La colonne 'DATE_FIN' correspond au Date de fin du couple code*libellé de commune (AAAA-MM-JJ). Lorsque cette colonne est vide, cela signifie que la commune est toujours active. <br>

In [5]:
df.head(10)

,COM,TNCC,NCC,NCCENR,LIBELLE,DATE_DEBUT,DATE_FIN
0,01001,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,1943-01-01,NaN
1,01002,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,1943-01-01,NaN
2,01003,1,AMAREINS,Amareins,Amareins,1943-01-01,1974-01-01
3,01004,1,AMBERIEU,Ambérieu,Ambérieu,1943-01-01,1955-03-31
4,01004,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1955-03-31,NaN
5,01005,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1943-01-01,NaN
6,01006,1,AMBLEON,Ambléon,Ambléon,1943-01-01,NaN
7,01007,1,AMBRONAY,Ambronay,Ambronay,1943-01-01,NaN
8,01008,1,AMBUTRIX,Ambutrix,Ambutrix,1943-01-01,NaN
9,01009,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,1943-01-01,NaN


## Inutile


In [ ]:
# Commencer par faire un DATAFRAME des communes qui ont été définitivement supprimées

df2 = df.copy() # DATAFRAME sans les communes qui ont été définitivement supprimées
supp = pd.DataFrame(columns = ['nom_commune','code_commune'],index = df.index.to_list())

k = 0
for i in tqdm(df.index.to_list()):
   if pd.notnull(df['DATE_FIN'][i]): # Cette ville n'existe plus
    autres = df[df['COM']==df['COM'][i]]
    #print(autres.shape[0])
    if autres.shape[0]==1:
      supp['nom_commune'][k] = df['NCC'][i]
      supp['code_commune'][k] = df['COM'][i]
      df2 = df2.drop(i)
      k += 1

    elif autres.shape[0]>1:
      autres = autres.drop(i)
      indices = autres.index.to_list()
      for l in indices:
        if pd.notnull(autres['DATE_FIN'][l]):
          supp['nom_commune'][k] = df['NCC'][l]
          supp['code_commune'][k] = df['COM'][l]
          k +=1
          if l in df2.index.to_list():
            df2 = df2.drop(l)

supp = supp.dropna()

100%|██████████| 42198/42198 [01:22<00:00, 509.64it/s] 


In [ ]:
print("Il y a ",supp.shape[0], " communes qui ont été supprimées. En voici un apperçu :")
print(supp.head(10))

Il y a  4845  communes qui ont été supprimées. En voici un apperçu :
                nom_commune code_commune
0                  AMAREINS        01003
1                     ARLOD        01018
2                    ARNANS        01020
3  BELLEGARDE SUR VALSERINE        01033
4                BELLEGARDE        01033
5         BELMONT LUTHEZIEU        01036
6                   BELMONT        01036
7                      BEON        01039
8                     BOHAS        01048
9                   BOUVENT        01055


In [ ]:
# Établir un DATAFRAME des communes ayant changé de nom et étant encore existante

transformation = pd.DataFrame(columns = ['ancien_nom','nouveau_nom','code_commune'],index = df.index.to_list())
k = 0
for i in tqdm(df2.index.to_list()):
  if pd.notnull(df2['DATE_FIN'][i]): # Cette ville n'existe plus

    # Trouver le nouveau nom :
    nouveaux_noms = df2[df2['COM']==df2['COM'][i]]
    # ! Il peut y en avoir plusieurs
    # C'est le cas où il y a plusieurs changements
    # Après avoir fait tourné un code simple, on apprend qu'il y a 474 cas
    # Il nous suffit de récupérer la liste des anciens noms et de mettre le nouveau nom
    nv = nouveaux_noms[nouveaux_noms['DATE_FIN'].isnull()]['NCC'].to_list()[0]
    indice = nouveaux_noms[nouveaux_noms['DATE_FIN'].isnull()].index.to_list()
    nouveaux_noms = nouveaux_noms.drop(indice)
    for j in nouveaux_noms.index.to_list():
      transformation['ancien_nom'][k] = nouveaux_noms['NCC'][j]
      transformation['code_commune'][k] = nouveaux_noms['COM'][j]
      transformation['nouveau_nom'][k] = nv
      k +=1

transformation = transformation.dropna()

100%|██████████| 37415/37415 [00:18<00:00, 2066.07it/s]


In [ ]:
print("Il y a ",transformation.shape[0], " communes dont le nom a changé.")


Il y a  2470  communes dont le nom a changé.


In [ ]:
transformation.head(15)

,ancien_nom,nouveau_nom,code_commune
0,AMBERIEU,AMBERIEU EN BUGEY,01004
1,ARBIGNIEU,ARBOYS EN BUGEY,01015
2,ARS,ARS SUR FORMANS,01021
3,BAGE LA VILLE,BAGE DOMMARTIN,01025
4,BELIGNAT,BELLIGNAT,01031
5,BOURG,BOURG EN BRESSE,01053
6,CHALLES,CHALLES LA MONTAGNE,01077
7,CHAMPAGNE,CHAMPAGNE EN VALROMEY,01079
8,CHAMPDOR,CHAMPDOR CORCELLES,01080
9,CHARNOZ,CHARNOZ SUR AIN,01088


In [ ]:
#transformation.to_csv('liste_transformations_communes.csv')
#supp.to_csv('liste_suppression_communes.csv')

In [ ]:
supp.head(15)

,nom_commune,code_commune
0,AMAREINS,01003
1,ARLOD,01018
2,ARNANS,01020
3,BELLEGARDE SUR VALSERINE,01033
4,BELLEGARDE,01033
5,BELMONT LUTHEZIEU,01036
6,BELMONT,01036
7,BEON,01039
8,BOHAS,01048
9,BOUVENT,01055


In [ ]:
# On va maintenant rechercher les communes qui ont changé de code_commune
travail = df[df.duplicated(['NCC'], keep = False)].sort_values(by = 'NCC')

In [ ]:
to_drop = []
for i in travail.index.to_list():
  nom = travail['NCC'][i]
  sub_df = travail[travail['NCC']==nom]
  indices = sub_df.index.to_list()
  longueur = len(indices)
  compteur = 0
  for k in indices:
    if pd.isnull(sub_df['DATE_FIN'][k]):
      compteur +=1
  if compteur == longueur:
    to_drop.append(indices)

drop = []
for sublist in to_drop:
    drop.extend(sublist)
drop = list(set(drop))

In [ ]:
travail = travail.drop(drop)

# Évènements sur les communes

Modalités pour le type d’événement <br>
 <br>
Changement de nom	  10<br>
Création	20<br>
Rétablissement	21<br>
Suppression	30<br>
Fusion simple	31<br>
Création de commune nouvelle	32<br>
Fusion association	33<br>
Transformation de fusion association en fusion simple	34<br>
Suppression de commune déléguée	35<br>
Changement de code dû à un changement de département	41<br>
Changement de code dû à un transfert de chef-lieu	50<br>
Transformation de commune associée en commune déléguée	70<br>
<br>
<br>
MOD	:	Type d'évènement de communes <br>
DATE_EFF	:	Date d'effet (AAAA-MM-JJ) <br>
COM_AV	:	Code de la commune avant évènement <br>
NCC_AV	:	Nom en clair (majuscules) <br>
TYPECOM_AP	:	Type de commune après l'évènement <br>
COM_AP	:	Code de la commune après l'évènement <br>
NCC_AP	:	Nom en clair (majuscules) <br>

In [80]:
# Importer les fichiers

ev = pd.read_csv('v_mvtcommune_2023.csv')
ev = ev.drop(['indic_oe', 'indic_oe₂','NCCENR_AV', 'LIBELLE_AV','TNCC_AV', 'TNCC_AP','NCCENR_AP', 'LIBELLE_AP'], axis=1)
ev = ev.sort_values(by = 'DATE_EFF')
print("Taille du fichier : ",ev.shape[0])
print("Colonnes que nous avons conservé : ", list(ev.columns))

Taille du fichier :  13834
Colonnes que nous avons conservé :  ['MOD', 'DATE_EFF', 'TYPECOM_AV', 'COM_AV', 'NCC_AV', 'TYPECOM_AP', 'COM_AP', 'NCC_AP']


In [133]:
# Créer des identifiants en faisant attention aux potentiels doublons

ev = ev.dropna(subset = ['COM_AV','NCC_AV'],axis=0)


ev['DEP_AV'] = ev['COM_AV'].apply(departement)
ev['DEP_AP'] = ev['COM_AP'].apply(departement)

liste_nom_ville_unique = list(set(ev['NCC_AV'].to_list()))

identifiant = 0 # création de l'identifiant
ev['id'] = 0 # création de la colonne
for ville in tqdm(liste_nom_ville_unique):
  identifiant +=1
  indices1 = ev[ev['NCC_AV']==ville].index.to_list()
  dpt1 = ev[ev['NCC_AV']==ville]['DEP_AV'].to_list()
  indices2 = ev[ev['NCC_AP']==ville].index.to_list()
  dpt2 = ev[ev['NCC_AV']==ville]['DEP_AP'].to_list()
  indices = list(set(indices1+indices2)) # liste des indices où le nom ville apparaît
  liste_dpt = list(set(dpt1+dpt2))
  if len(liste_dpt)==1:
    for i in indices:
      ev['id'][i] = identifiant

  else:
    dep_set = liste_dpt[0]

    for i in indices:
      if ev['DEP_AV'][i]==ev['DEP_AP'][i]:
        if ev['DEP_AV'][i]==dep_set:
          ev['id'][i] = identifiant
          dep_set = ev['DEP_AV'][i]
        else:
          identifiant += 1
          ev['id'][i] = identifiant
          dep_set = ev['DEP_AV'][i]

      else:
        if ev['DEP_AV'][i]==dep_set:
          ev['id'][i] = identifiant
          dep_set = ev['DEP_AP'][i]
        else:
          identifiant +=1
          ev['id'][i] = identifiant
          dep_set = ev['DEP_AP'][i]


ev = ev.sort_values(by = ['id','DATE_EFF'])

<ipython-input-133-8b4810739c14>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev['id'][i] = identifiant
<ipython-input-133-8b4810739c14>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev['id'][i] = identifiant
<ipython-input-133-8b4810739c14>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev['id'][i] = identifiant
<ipython-input-133-8b4810739c14>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [135]:
# Regarder s'il y a des problemes pour les villes doublons

id_unique = [0]
liste = ev['id'].to_list()
for indice in liste:
  if liste.count(indice)==1:
    id_unique.append(indice)

sub_ev = ev[~ev['id'].isin(id_unique)]

def departement(row):
  if pd.notnull(row):
    return row[0:2]

sub_ev['DEP_AV'] = sub_ev['COM_AV'].apply(departement)
sub_ev['DEP_AP'] = sub_ev['COM_AP'].apply(departement)

pb = sub_ev.copy()
adrop = []
for k in sub_ev['id'].to_list():
  subsub_ev = sub_ev[sub_ev['id']==k]
  index = subsub_ev.index.to_list()
  liste = list(set(subsub_ev['DEP_AV'].to_list()+subsub_ev['DEP_AP'].to_list()))
  if len(liste)==1:
    adrop.append(index)

a = []
for sous_liste in adrop:
  a = a+sous_liste

a = list(set(a))
pb = pb.drop(a)


id_restant = pb['id'].to_list()
adrop = []
for i in id_restant:
  d = pb[pb['id']==i]
  liste = list(set(d['DEP_AV'].to_list()+d['DEP_AP'].to_list()))
  index = d.index.to_list()
  for k in index:
    if d['DEP_AV'][k]!=d['DEP_AP'][k]:
      if len(liste)==2:
        adrop.append(index)

a = []
for sous_liste in adrop:
  a = a+sous_liste

a = list(set(a))
pb = pb.drop(a)

<ipython-input-135-4b1e0c83c99b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_ev['DEP_AV'] = sub_ev['COM_AV'].apply(departement)
<ipython-input-135-4b1e0c83c99b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_ev['DEP_AP'] = sub_ev['COM_AP'].apply(departement)


In [140]:
ev

,MOD,DATE_EFF,TYPECOM_AV,COM_AV,NCC_AV,TYPECOM_AP,COM_AP,NCC_AP,id,DEP_AV,DEP_AP
12366,31,1965-03-01,COM,27096,BOTTEREAUX,COM,27096,BOTTEREAUX,1,27,27
3266,32,2016-12-15,COM,49233,NYOISEAU,COMD,49233,NYOISEAU,2,49,49
3267,32,2016-12-15,COM,49233,NYOISEAU,COM,49331,SEGRE EN ANJOU BLEU,2,49,49
6732,10,1987-02-06,COM,73116,FONTCOUVERTE,COM,73116,FONTCOUVERTE LA TOUSSUIRE,3,73,73
3933,32,2016-01-01,COM,27112,BRETEUIL,COM,27112,BRETEUIL,4,27,27
...,...,...,...,...,...,...,...,...,...,...,...
12718,31,1962-12-20,COM,04046,CHAFFAUT LAGREMUSE,COM,04046,CHAFFAUT SAINT JURSON,8712,04,04
11989,41,1968-01-01,COM,78428,MONTMORENCY,COM,95428,MONTMORENCY,8713,78,95
13354,20,1955-06-20,COM,31232,GRENADE,COM,31232,GRENADE,8714,31,31
13355,20,1955-06-20,COM,31232,GRENADE,COM,31592,LARRA,8714,31,31


In [158]:
from pandas.core.groupby.base import dataclasses
df = pd.DataFrame(columns = ['ville43','ville(s)_inter','code(s)_inter','ville23','code43','code23','changements'], index = list(range(7529)))

for i in list(range(1,7530)):
  sub_ev = ev[ev['id']==i]
  indices = sub_ev.index.to_list()


  if len(indices)==1:
    l = indices[0]
    df['ville23'][i] = sub_ev['NCC_AP'][l]
    df['ville43'][i] = sub_ev['NCC_AV'][l]
    df['code23'][i] = sub_ev['COM_AP'][l]
    df['code43'][i] = sub_ev['COM_AV'][l]
    df['ville(s)_inter'][i] = np.nan
    df['code(s)_inter'][i] = np.nan
    df['changements'][i] = sub_ev['MOD'][l]


  elif len(indices)>1:
    change = sub_ev['MOD'].to_list()
    changement = ""
    for c in change:
      c = str(c)
      changement = changement+c+"*"
    n = len(changement) - 1
    changement = changement[:n]
    min = indices[0]
    max = indices[-1]
    df['ville23'][i] = sub_ev['NCC_AP'][max]
    df['ville43'][i] = sub_ev['NCC_AV'][min]
    df['code23'][i] = sub_ev['COM_AP'][max]
    df['code43'][i] = sub_ev['COM_AV'][min]
    indices = [x for x in indices if x != min]
    indices = [x for x in indices if x != max]
    vill = ""
    cod = ""
    for k in indices:
      vill = vill+sub_ev['NCC_AV'][k]+"*"
      cod = cod+sub_ev['COM_AV'][k]+"*"

    n = len(vill) - 1
    vill = vill[:n]
    n = len(cod) - 1
    cod = cod[:n]
    df['ville(s)_inter'][i] = vill
    df['code(s)_inter'][i] = cod
    df['changements'][i] = changement


df = df.dropna(how='all')
df = df.reset_index(drop=True)

In [160]:
df.to_csv('evenement_commune.csv')

# Travail sur les deux fichiers

In [56]:
df1 = pd.read_csv('v_commune_depuis_1943.csv')
df2 = pd.read_csv('evenement_communes.csv')

In [57]:
df2 = df2.drop(['DATE_FIN','DATE_DEBUT'], axis=1)

In [77]:
df = df2.copy()
df['DATE_DEBUT'] = np.nan
df['DATE_FIN'] = np.nan

for i in tqdm(df.index.to_list()):

  if df['DEP43'][i]=='20':
    sub_df_avant = df1[df1['COM']==df['code43'][i]]
    sub_df_apres = df1[df1['COM']==df['code23'][i]]

    if sub_df_avant['DATE_DEBUT'].shape[0]>0:
      df['DATE_DEBUT'][i] = sub_df_avant['DATE_DEBUT'].to_list()[0]

    if sub_df_apres['DATE_FIN'].shape[0]>0:
      df['DATE_FIN'][i] = sub_df_apres['DATE_FIN'].to_list()[0]

  else:
    sub_df_avant = df1[df1['NCC']==df['ville43'][i]]
    sub_df_apres = df1[df1['NCC']==df['ville23'][i]]

    if sub_df_avant['DATE_DEBUT'].shape[0]>0:
      df['DATE_DEBUT'][i] = sub_df_avant['DATE_DEBUT'].to_list()[0]

    if sub_df_apres['DATE_FIN'].shape[0]>0:
      df['DATE_FIN'][i] = sub_df_apres['DATE_FIN'].to_list()[0]




df.head(10)

  0%|          | 0/5336 [00:00<?, ?it/s]<ipython-input-77-aeed8752b3c2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_DEBUT'][i] = sub_df_avant['DATE_DEBUT'].to_list()[0]
<ipython-input-77-aeed8752b3c2>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_FIN'][i] = sub_df_apres['DATE_FIN'].to_list()[0]
100%|██████████| 5336/5336 [01:07<00:00, 79.61it/s]


,Unnamed: 0,ville43,ville(s)_inter,code(s)_inter,ville23,code43,code23,changements,DEP43,DEP23,DATE_DEBUT,DATE_FIN
0,0,BOTTEREAUX,NaN,NaN,BOTTEREAUX,27096,27096,31,27,27,1943-01-01,NaN
1,1,NYOISEAU,NaN,NaN,SEGRE EN ANJOU BLEU,49233,49331,32*32,49,49,1943-01-01,NaN
2,2,FONTCOUVERTE,NaN,NaN,FONTCOUVERTE LA TOUSSUIRE,73116,73116,10,73,73,1943-01-01,NaN
3,3,BRETEUIL,NaN,NaN,BRETEUIL,27112,27112,32*32,27,27,1943-01-01,NaN
4,4,CHENE CHENU,CHENE CHENU,28097,TREMBLAY LES VILLAGES,28097,28393,33*33*34,28,28,1943-01-01,NaN
5,5,VOUVRAY,NaN,NaN,VOUVRAY,01458,01458,33,01,01,1943-01-01,1973-11-01
6,6,BLERANCOURT,NaN,NaN,BLERANCOURT,02093,02093,31,02,02,1943-01-01,NaN
7,7,TARDIERE,NaN,NaN,TERVAL,85289,85289,32*32,85,85,1943-01-01,NaN
8,8,BOSSELSHAUSEN,NaN,NaN,BOSSELSHAUSEN,67057,67057,33*21,67,67,1943-01-01,1974-03-01
9,9,BOURGVALLEES,NaN,NaN,BOURGVALLEES,50546,50546,32,50,50,2016-01-01,NaN


In [88]:
index_fin = df[pd.notnull(df['DATE_FIN'])].index.to_list()
liste_ville_43 = df['ville43'].to_list()
liste_code_43 = df['code43'].to_list()

for k in index_fin:
  ville_fin = df['ville23'][k]
  code_fin = df['code23'][k]
  if ville_fin in liste_ville_43:
    df = df.drop(k)

In [94]:
df['DATE_FIN'].count()

140

In [104]:
df1[df1['NCC']=='MONCEAUX AU PERCHE']

,COM,TNCC,NCC,NCCENR,LIBELLE,DATE_DEBUT,DATE_FIN
28264,61280,0,MONCEAUX AU PERCHE,Monceaux-au-Perche,Monceaux-au-Perche,1992-09-01,2016-01-01


In [99]:
df.to_csv('evenements_communes.csv')

### Apparté

In [40]:
# Encore du chemin à parcourir : finir le parcours de certaine ville !

sub_df = df[pd.notnull(df['DATE_FIN'])]
index_pb = sub_df.index.to_list()

liste_ville = sub_df['ville23'].to_list()
liste_ville2 = df['ville43'].to_list()


for k in index_pb:
  ville = sub_df['ville23'][k]
  if ville in liste_ville2 :
    df = df.drop(k)

df

,Unnamed: 0,ville43,ville(s)_inter,code(s)_inter,ville23,code43,code23,changements,DATE_DEBUT,DATE_FIN
0,0,BOTTEREAUX,NaN,NaN,BOTTEREAUX,27096,27096,31,1943-01-01,NaN
1,1,NYOISEAU,NaN,NaN,SEGRE EN ANJOU BLEU,49233,49331,32*32,1943-01-01,NaN
2,2,FONTCOUVERTE,NaN,NaN,FONTCOUVERTE LA TOUSSUIRE,73116,73116,10,NaN,NaN
3,3,BRETEUIL,NaN,NaN,BRETEUIL,27112,27112,32*32,NaN,NaN
4,4,CHENE CHENU,CHENE CHENU,28097,TREMBLAY LES VILLAGES,28097,28393,33*33*34,1943-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...
6497,6497,ROCHE GUYON,NaN,NaN,ROCHE GUYON,78523,95523,41,NaN,NaN
6498,6498,SERRA DI FIUMORBO,NaN,NaN,SERRA DI FIUMORBO,20277,2B277,41,NaN,NaN
6500,6500,VENTENAC D AUDE,NaN,NaN,VENTENAC EN MINERVOIS,11405,11405,10,1943-01-01,NaN
6501,6501,LOISEY,CULEY*LOISEY CULEY,55138*55298,CULEY,55298,55138,31*31*21*21,NaN,NaN


In [47]:
df = df.drop('Unnamed: 0', axis = 1)

In [50]:
df = df.reset_index(drop=True)
def departement(row):
  if pd.notnull(row):
    return row[0:2]

df['DEP43'] = df['code43'].apply(departement)
df['DEP23'] = df['code23'].apply(departement)

df.to_csv('evenement_communes.csv')

In [54]:
df[df['DEP43']=='75']

,ville43,ville(s)_inter,code(s)_inter,ville23,code43,code23,changements,DATE_DEBUT,DATE_FIN,DEP43,DEP23
100,PLESSIS ROBINSON,NaN,NaN,PLESSIS ROBINSON,75060,92060,41,NaN,NaN,75,92
119,NOISY LE SEC,NaN,NaN,NOISY LE SEC,75053,93053,41,NaN,NaN,75,93
133,ORLY,NaN,NaN,ORLY,75054,94054,41,NaN,NaN,75,94
254,PAVILLONS SOUS BOIS,NaN,NaN,PAVILLONS SOUS BOIS,75057,93057,41,NaN,NaN,75,93
293,VILLENEUVE LA GARENNE,NaN,NaN,VILLENEUVE LA GARENNE,75078,92078,41,NaN,NaN,75,92
...,...,...,...,...,...,...,...,...,...,...,...
4616,SURESNES,NaN,NaN,SURESNES,75073,92073,41,NaN,NaN,75,92
4625,SAINT MANDE,NaN,NaN,SAINT MANDE,75067,94067,41,NaN,NaN,75,94
4861,MONTROUGE,NaN,NaN,MONTROUGE,75049,92049,41,NaN,NaN,75,92
4923,CACHAN,NaN,NaN,CACHAN,75016,94016,41,NaN,NaN,75,94
